<a href="https://colab.research.google.com/github/han-jisu0525/Data-Analysis/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/lstm/TSLA.csv")
print("Numbers of rows and columns", df.shape)

Numbers of rows and columns (1258, 7)


In [3]:
df.head(5)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2018-12-31,22.519333,22.614000,21.684000,22.186666,22.186666,94534500
1,2019-01-02,20.406668,21.008667,19.920000,20.674667,20.674667,174879000
2,2019-01-03,20.466667,20.626667,19.825333,20.024000,20.024000,104478000
3,2019-01-04,20.400000,21.200001,20.181999,21.179333,21.179333,110911500
4,2019-01-07,21.448000,22.449333,21.183332,22.330667,22.330667,113268000


In [4]:
import numpy as np
test_set = df.iloc[:600, 1:2].values
training_set = df.iloc[600:, 1:2].values

In [5]:
# 데이터 정규화
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

In [6]:
X_train = []
y_train = []
for i in range(50, 658):
  X_train.append(training_set_scaled[i-50:i, 0])
  y_train.append(training_set_scaled[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [7]:
# 모델 만들기
from keras.models import Sequential
model = Sequential()

In [8]:
# 첫번째 레이어 + 정규화
from keras.layers import LSTM, Dropout
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))

In [9]:
# 2 ~ n-1번째 레이어 + 정규화
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))

In [10]:
# 마지막 레이어 + 정규화
model.add(LSTM(units=50))
model.add(Dropout(0.2))

# 출력 레이어 추가
from keras.layers import Dense
model.add(Dense(units=1))

In [11]:
# 모델 컴파일
model.compile(optimizer='adam', loss='mean_squared_error')

# 모델 학습
model.fit(X_train, y_train, epochs=100, batch_size=32)

Epoch 1/100
19/19 [==============================] - 8s 71ms/step - loss: 0.0804
Epoch 2/100
19/19 [==============================] - 1s 72ms/step - loss: 0.0211
Epoch 3/100
19/19 [==============================] - 1s 73ms/step - loss: 0.0151
Epoch 4/100
19/19 [==============================] - 2s 86ms/step - loss: 0.0139
Epoch 5/100
19/19 [==============================] - 2s 116ms/step - loss: 0.0130
Epoch 6/100
19/19 [==============================] - 2s 119ms/step - loss: 0.0122
Epoch 7/100
19/19 [==============================] - 2s 91ms/step - loss: 0.0102
Epoch 8/100
19/19 [==============================] - 1s 73ms/step - loss: 0.0105
Epoch 9/100
19/19 [==============================] - 1s 72ms/step - loss: 0.0111
Epoch 10/100
19/19 [==============================] - 1s 76ms/step - loss: 0.0092
Epoch 11/100
19/19 [==============================] - 1s 75ms/step - loss: 0.0096
Epoch 12/100
19/19 [==============================] - 1s 71ms/step - loss: 0.0090
Epoch 13/100
19/19 [===

In [12]:
dataset_train = df.iloc[600:, 1:2]
dataset_test = df.iloc[:600, 1:2]

dataset_total = pd.concat((dataset_train, dataset_test), axis = 0)

In [13]:
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 50:].values
inputs = inputs.reshape(-1, 1)
inputs = sc.transform(inputs)

In [14]:
X_test = []
for i in range(50, 510):
  X_test.append(inputs[i-50:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
print(X_test.shape)


(460, 50, 1)


In [15]:
predicted_stock_price = model.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

15/15 [==============================] - 2s 25ms/step
